# CMSC471 Artificial Intelligence - Fall 2020
## Instructor: Fereydoon Vafaei
# <font color=blue> Assignment 1: Solving Problems by Searching </font>

Sam Bailor - VK96692

## Overview and Learning Objectives

We've studied two types of search algorithms during the recent lectures: Uninformed Search and Informed Search.

Uninformed Search (aka blind search) refers to the search strategies that do not use additional information about states and goal of the search - nothing beyond the provided problem definition. The first algorithms that we studied from Chapter 3 of the Russel & Norvig textbook were Breadth First Search (BFS) and Depth First Search (DFS) which both fall into the category of Uniformed Search.

We also studied how we can modify DFS to combine the advantages of BFS (completeness and conditional optimality) with the main advantage of DFS --space complexity of $O(bm)$ where $b$ is the branching factor and $m$ is the maximum depth of the search space. Limited Depth Search and Iterative Deepening DFS were two algorithms that we studied in that section and you are going to use them in Part I of this assignment.

In Part II, you are going to practice Informed Search algorithms and heuristic functions. As discussed in the lectures, informed search algorithms use heuristic functions.

Finally, in Part III, you will work on an example graph by applying Informed Search algorithms: Greedy Best-First Search and A*. 

<b>Very Important Note:</b> Read ALL the instructions in this notebook very carefully. Careless reading and skipping lines would be a major source of making mistakes and losing points in your first assignment! Also notice that this assignment has three parts and includes multiple steps and questions. You're strongly recommended to get started early and plan to finish well before the due. Technical problems or other issues/questions on the due date or just a day before would NOT be accepted as an excuse to delay your submission. As stated in the policy, ALL assignments are indvidual work and students are strictly prohibited from collaboration on assignments. Students are responsible to debug the code and resolve any errors that may arise. Students should NOT share any answer, solution, or code/snippet in Piazza. Violations of these policies would be penalized accordingly.

Pedagogically, this assignment will help you:
- better understand how these algorithms work in pratice. 
- brush up your Python skills - and possibly learn a couple of new "Pythonic" tricks!
- pratice reading documentation. This is a very important skill in AI/ML/Data Science collaborative environments and teams.

So, let's get started!

# Part I - Uninformed Search

In <b>Part I</b> of this Jupyter Notebook, you are going to apply Uninformed Search algorithms on a couple of search problems.

First, you need to download the search notebook from AIMA GitHub repo. Make sure that it is the 4th edition:
https://github.com/aimacode/aima-python/blob/master/search4e.ipynb

You are going to use some of the classes and functions of `search4e.ipynb`. You can add cells and paste the borrowed code or a modified version of it.

**Note:** In this notebook, students are required to import any necessary class or function from `search4e.ipynb` and add/modify the code wherever it is needed. You are required to resolve any issues or errors that may arise to make all the imported codes as well as YOUR codes run error-free. 

## <font color=orange> Problems and Nodes

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import random
import heapq
import math
import sys
from collections import defaultdict, deque, Counter
from itertools import combinations

class Problem(object):
    """The abstract class for a formal problem. A new domain subclasses this,
    overriding `actions` and `results`, and perhaps other methods.
    The default heuristic is 0 and the default action cost is 1 for all states.
    When yiou create an instance of a subclass, specify `initial`, and `goal` states 
    (or give an `is_goal` method) and perhaps other keyword args for the subclass."""

    def __init__(self, initial=None, goal=None, **kwds): 
        self.__dict__.update(initial=initial, goal=goal, **kwds) 
        
    def actions(self, state):        raise NotImplementedError
    def result(self, state, action): raise NotImplementedError
    def is_goal(self, state):        return state == self.goal
    def action_cost(self, s, a, s1): return 1
    def h(self, node):               return 0
    
    def __str__(self):
        return '{}({!r}, {!r})'.format(
            type(self).__name__, self.initial, self.goal)
    

class Node:
    "A Node in a search tree."
    def __init__(self, state, parent=None, action=None, path_cost=0):
        self.__dict__.update(state=state, parent=parent, action=action, path_cost=path_cost)

    def __repr__(self): return '<{}>'.format(self.state)
    def __len__(self): return 0 if self.parent is None else (1 + len(self.parent))
    def __lt__(self, other): return self.path_cost < other.path_cost
    
    
failure = Node('failure', path_cost=math.inf) # Indicates an algorithm couldn't find a solution.
cutoff  = Node('cutoff',  path_cost=math.inf) # Indicates iterative deepening search was cut off.
    
    
def expand(problem, node):
    "Expand a node, generating the children nodes."
    s = node.state
    for action in problem.actions(s):
        s1 = problem.result(s, action)
        cost = node.path_cost + problem.action_cost(s, action, s1)
        yield Node(s1, node, action, cost)
        

def path_actions(node):
    "The sequence of actions to get to this node."
    if node.parent is None:
        return []  
    return path_actions(node.parent) + [node.action]


def path_states(node):
    "The sequence of states to get to this node."
    if node in (cutoff, failure, None): 
        return []
    return path_states(node.parent) + [node.state]

## <font color=orange> Queues

In [2]:
FIFOQueue = deque

LIFOQueue = list

class PriorityQueue:
    """A queue in which the item with minimum f(item) is always popped first."""

    def __init__(self, items=(), key=lambda x: x): 
        self.key = key
        self.items = [] # a heap of (score, item) pairs
        for item in items:
            self.add(item)
         
    def add(self, item):
        """Add item to the queuez."""
        pair = (self.key(item), item)
        heapq.heappush(self.items, pair)

    def pop(self):
        """Pop and return the item with min f(item) value."""
        return heapq.heappop(self.items)[1]
    
    def top(self): return self.items[0][1]

    def __len__(self): return len(self.items)

## <font color=orange> Search Algorithms: Best-First

In [3]:
def best_first_search(problem, f):
    "Search nodes with minimum f(node) value first."
    node = Node(problem.initial)
    frontier = PriorityQueue([node], key=f)
    reached = {problem.initial: node}
    while frontier:
        node = frontier.pop()
        if problem.is_goal(node.state):
            return node
        for child in expand(problem, node):
            s = child.state
            if s not in reached or child.path_cost < reached[s].path_cost:
                reached[s] = child
                frontier.add(child)
    return failure


def best_first_tree_search(problem, f):
    "A version of best_first_search without the `reached` table."
    frontier = PriorityQueue([Node(problem.initial)], key=f)
    while frontier:
        node = frontier.pop()
        if problem.is_goal(node.state):
            return node
        for child in expand(problem, node):
            if not is_cycle(child):
                frontier.add(child)
    return failure


def g(n): return n.path_cost


def astar_search(problem, h=None):
    """Search nodes with minimum f(n) = g(n) + h(n)."""
    h = h or problem.h
    return best_first_search(problem, f=lambda n: g(n) + h(n))


def astar_tree_search(problem, h=None):
    """Search nodes with minimum f(n) = g(n) + h(n), with no `reached` table."""
    h = h or problem.h
    return best_first_tree_search(problem, f=lambda n: g(n) + h(n))


def weighted_astar_search(problem, h=None, weight=1.4):
    """Search nodes with minimum f(n) = g(n) + weight * h(n)."""
    h = h or problem.h
    return best_first_search(problem, f=lambda n: g(n) + weight * h(n))

        
def greedy_bfs(problem, h=None):
    """Search nodes with minimum h(n)."""
    h = h or problem.h
    return best_first_search(problem, f=h)


def uniform_cost_search(problem):
    "Search nodes with minimum path cost first."
    return best_first_search(problem, f=g)


def breadth_first_bfs(problem):
    "Search shallowest nodes in the search tree first; using best-first."
    return best_first_search(problem, f=len)


def depth_first_bfs(problem):
    "Search deepest nodes in the search tree first; using best-first."
    return best_first_search(problem, f=lambda n: -len(n))


def is_cycle(node, k=30):
    "Does this node form a cycle of length k or less?"
    def find_cycle(ancestor, k):
        return (ancestor is not None and k > 0 and
                (ancestor.state == node.state or find_cycle(ancestor.parent, k - 1)))
    return find_cycle(node.parent, k)

## <font color=orange> Other Search Algorithms

In [4]:
def breadth_first_search(problem):
    "Search shallowest nodes in the search tree first."
    node = Node(problem.initial)
    if problem.is_goal(problem.initial):
        return node
    frontier = FIFOQueue([node])
    reached = {problem.initial}
    while frontier:
        node = frontier.pop()
        for child in expand(problem, node):
            s = child.state
            if problem.is_goal(s):
                return child
            if s not in reached:
                reached.add(s)
                frontier.appendleft(child)
    return failure


def iterative_deepening_search(problem):
    "Do depth-limited search with increasing depth limits."
    for limit in range(1, sys.maxsize):
        result = depth_limited_search(problem, limit)
        if result != cutoff:
            return result
        
        
def depth_limited_search(problem, limit=10):
    "Search deepest nodes in the search tree first."
    frontier = LIFOQueue([Node(problem.initial)])
    result = failure
    while frontier:
        node = frontier.pop()
        if problem.is_goal(node.state):
            return node
        elif len(node) >= limit:
            result = cutoff
        elif not is_cycle(node):
            for child in expand(problem, node):
                frontier.append(child)
    return result


def depth_first_recursive_search(problem, node=None):
    if node is None: 
        node = Node(problem.initial)
    if problem.is_goal(node.state):
        return node
    elif is_cycle(node):
        return failure
    else:
        for child in expand(problem, node):
            result = depth_first_recursive_search(problem, child)
            if result:
                return result
        return failure

## Route Finding Problem

Using `RouteProblem` and `Map` classes, define a few new problems as described below called `r5`, `r6`, `r7` that represent different routes on the following **directed** graph. Notice that the `Map` must be defined as **directed**.

**Hint**: See how Romania map is defined in `search4e.ipynb`; however, you don't need locations in this case. Also, all the costs are assumed to be identical (the default cost value is 1).

The `A1-graph` is illustrated below.

<img src="A1-graph.png" align="left"/>

## <font color=orange> Route Finding Dependencies

In [5]:
class RouteProblem(Problem):
    """A problem to find a route between locations on a `Map`.
    Create a problem with RouteProblem(start, goal, map=Map(...)}).
    States are the vertexes in the Map graph; actions are destination states."""
    
    def actions(self, state): 
        """The places neighboring `state`."""
        return self.map.neighbors[state]
    
    def result(self, state, action):
        """Go to the `action` place, if the map says that is possible."""
        return action if action in self.map.neighbors[state] else state
    
    def action_cost(self, s, action, s1):
        """The distance (cost) to go from s to s1."""
        return self.map.distances[s, s1]
    
    def h(self, node):
        "Straight-line distance between state and the goal."
        locs = self.map.locations
        return straight_line_distance(locs[node.state], locs[self.goal])
    
    
def straight_line_distance(A, B):
    "Straight-line distance between two points."
    return sum(abs(a - b)**2 for (a, b) in zip(A, B)) ** 0.5

In [6]:
class Map:
    """A map of places in a 2D world: a graph with vertexes and links between them. 
    In `Map(links, locations)`, `links` can be either [(v1, v2)...] pairs, 
    or a {(v1, v2): distance...} dict. Optional `locations` can be {v1: (x, y)} 
    If `directed=False` then for every (v1, v2) link, we add a (v2, v1) link."""

    def __init__(self, links, locations=None, directed=False):
        if not hasattr(links, 'items'): # Distances are 1 by default
            links = {link: 1 for link in links}
        if not directed:
            for (v1, v2) in list(links):
                links[v2, v1] = links[v1, v2]
        self.distances = links
        self.neighbors = multimap(links)
        self.locations = locations or defaultdict(lambda: (0, 0))
        
def multimap(pairs) -> dict:
    "Given (key, val) pairs, make a dict of {key: [val,...]}."
    result = defaultdict(list)
    for key, val in pairs:
        result[key].append(val)
    return result

## <font color=green> Route Finding Tests

In [7]:
A1_graph = Map(
    {('a','b'):1, ('a','c'):1, ('a','d'):1, ('b','e'):1, ('b','f'):1, ('b','g'):1, ('c','h'):1,
     ('c','i'):1, ('d','j'):1, ('d','k'):1, ('e','l'):1, ('e','m'):1, ('f','n'):1, ('f','o'):1,
     ('g','p'):1, ('g','q'):1, ('l','r'):1, ('l','s'):1, ('r','t'):1, ('r','u'):1}, None, True)

> `r5` should be defined from `a` to `m` <br>
> `r6` should be defined from `a` to `u` <br>
> `r7` should be defined from `b` to `d`

In [8]:
r5 = RouteProblem('a', 'm', map = A1_graph)

r6 = RouteProblem('a', 'u', map = A1_graph)

r7 = RouteProblem('b', 'd', map = A1_graph)

Now, let's try DFS with some example tests on this graph. The correct output is provided for your reference.

In [9]:
path_states(depth_first_bfs(r5))

['a', 'b', 'e', 'm']

In [10]:
path_states(depth_first_bfs(r6))

['a', 'b', 'e', 'l', 'r', 'u']

>What if no path exists?! Let's try!

In [11]:
# Non-existing path!
path_states(depth_first_bfs(r7))

[]

Now, try the tests on `r5` and `r6` with Depth-limited algorithm (with a depth limit of 3) and Iterative-deepening search. Outputs are not provided for these tests.

In [12]:
path_states(depth_limited_search(r5, limit=3))

['a', 'b', 'e', 'm']

In [21]:
path_states(depth_limited_search(r6, limit=3))

[]

In [14]:
path_states(iterative_deepening_search(r5))

['a', 'b', 'e', 'm']

In [15]:
path_states(iterative_deepening_search(r6))

['a', 'b', 'e', 'l', 'r', 'u']

## Grid problem

Now, try Breadth-first search (`breadth_first_bfs`), DFS (`depth_first_bfs`), Depth-Limited with a default value of the limit, and Iterative Deepening on `d1` grid problem from `search4e.ipynb`.

You can use `path_states()` function for the grid problem too. If any runs of the algorithms takes more than 15 minutes on your machine, you may stop the kernel, leave a comment in your code indicating what happened, and move on.

## <font color=orange> Grid Problem Dependencies

In [9]:
class GridProblem(Problem):
    """Finding a path on a 2D grid with obstacles. Obstacles are (x, y) cells."""

    def __init__(self, initial=(15, 30), goal=(130, 30), obstacles=(), **kwds):
        Problem.__init__(self, initial=initial, goal=goal, 
                         obstacles=set(obstacles) - {initial, goal}, **kwds)

    directions = [(-1, -1), (0, -1), (1, -1),
                  (-1, 0),           (1,  0),
                  (-1, +1), (0, +1), (1, +1)]
    
    def action_cost(self, s, action, s1): return straight_line_distance(s, s1)
    
    def h(self, node): return straight_line_distance(node.state, self.goal)
                  
    def result(self, state, action): 
        "Both states and actions are represented by (x, y) pairs."
        return action if action not in self.obstacles else state
    
    def actions(self, state):
        """You can move one cell in any of `directions` to a non-obstacle cell."""
        x, y = state
        return {(x + dx, y + dy) for (dx, dy) in self.directions} - self.obstacles
    
class ErraticVacuum(Problem):
    def actions(self, state): 
        return ['suck', 'forward', 'backward']
    
    def results(self, state, action): return self.table[action][state]
    
    table = dict(suck=    {1:{5,7}, 2:{4,8}, 3:{7}, 4:{2,4}, 5:{1,5}, 6:{8}, 7:{3,7}, 8:{6,8}},
                 forward= {1:{2}, 2:{2}, 3:{4}, 4:{4}, 5:{6}, 6:{6}, 7:{8}, 8:{8}},
                 backward={1:{1}, 2:{1}, 3:{3}, 4:{3}, 5:{5}, 6:{5}, 7:{7}, 8:{7}})
    
# Some grid routing problems

# The following can be used to create obstacles:
    
def random_lines(X=range(15, 130), Y=range(60), N=150, lengths=range(6, 12)):
    """The set of cells in N random lines of the given lengths."""
    result = set()
    for _ in range(N):
        x, y = random.choice(X), random.choice(Y)
        dx, dy = random.choice(((0, 1), (1, 0)))
        result |= line(x, y, dx, dy, random.choice(lengths))
    return result

def line(x, y, dx, dy, length):
    """A line of `length` cells starting at (x, y) and going in (dx, dy) direction."""
    return {(x + i * dx, y + i * dy) for i in range(length)}

random.seed(42) # To make this reproducible

frame = line(-10, 20, 0, 1, 20) | line(150, 20, 0, 1, 20)
cup = line(102, 44, -1, 0, 15) | line(102, 20, -1, 0, 20) | line(102, 44, 0, -1, 24)

d1 = GridProblem(obstacles=random_lines(N=100) | frame)

## <font color=green> Grid Problem Tests

In [17]:
path_states(breadth_first_bfs(d1))

[(15, 30),
 (16, 30),
 (17, 30),
 (18, 30),
 (19, 31),
 (20, 32),
 (21, 33),
 (22, 33),
 (23, 34),
 (24, 35),
 (25, 36),
 (26, 36),
 (27, 36),
 (28, 36),
 (29, 36),
 (30, 36),
 (31, 36),
 (32, 37),
 (33, 38),
 (34, 39),
 (35, 40),
 (36, 40),
 (37, 40),
 (38, 40),
 (39, 40),
 (40, 40),
 (41, 40),
 (42, 40),
 (43, 40),
 (44, 40),
 (45, 40),
 (46, 40),
 (47, 40),
 (48, 40),
 (49, 40),
 (50, 40),
 (51, 40),
 (52, 40),
 (53, 40),
 (54, 40),
 (55, 40),
 (56, 40),
 (57, 40),
 (58, 40),
 (59, 40),
 (60, 40),
 (61, 40),
 (62, 40),
 (63, 40),
 (64, 40),
 (65, 40),
 (66, 40),
 (67, 39),
 (68, 38),
 (69, 37),
 (70, 36),
 (71, 35),
 (72, 34),
 (73, 33),
 (74, 32),
 (75, 32),
 (76, 31),
 (77, 31),
 (78, 30),
 (79, 29),
 (80, 28),
 (81, 27),
 (82, 27),
 (83, 27),
 (84, 27),
 (85, 27),
 (86, 27),
 (87, 27),
 (88, 27),
 (89, 28),
 (90, 28),
 (91, 28),
 (92, 28),
 (93, 28),
 (94, 28),
 (95, 28),
 (96, 28),
 (97, 28),
 (98, 28),
 (99, 28),
 (100, 28),
 (101, 28),
 (102, 28),
 (103, 28),
 (104, 28),
 (105

In [18]:
path_states(depth_first_bfs(d1))

# RecursionError

RecursionError: maximum recursion depth exceeded

In [19]:
path_states(depth_limited_search(d1))

[]

In [ ]:
path_states(iterative_deepening_search(d1))

#Exceeded 15 minutes

> Next, define a new grid problem named `d8` from `(4,4)` to `(8,3)` with no obstacles, and with the default directions, and run all the aforementioned algorithms BFS, DFS (this time the other version i.e. `depth_first_recursive_search`), depth-limited with default limit and iterative-deepening on it.

In [19]:
d8 = GridProblem((4,4), (8,3))

In [20]:
path_states(breadth_first_bfs(d8))

[(4, 4), (5, 4), (6, 4), (7, 4), (8, 3)]

In [21]:
path_states(depth_first_recursive_search(d8))

[(4, 4), (5, 4), (6, 4), (7, 3), (8, 3)]

In [22]:
path_states(depth_limited_search(d8))

[(4, 4),
 (5, 3),
 (5, 2),
 (4, 1),
 (4, 0),
 (4, -1),
 (5, 0),
 (5, 1),
 (6, 2),
 (7, 2),
 (8, 3)]

In [23]:
path_states(iterative_deepening_search(d8))

[(4, 4), (5, 3), (6, 2), (7, 2), (8, 3)]

## Part I Questions

- Q1 [2 points] - What is the minimum required depth limit for Depth-limited search so that it can return the desired output for `r6`?


- Q2 [4 points] - Which of the algorithms that you tried returned a solution path for `d1`?

- Q3 [4 points] - Which of the algorithms that you tried returned a solution path for `d8`?

<font color=red>Enter your answers in the following markdown cell.</font>

- Your answers to Part I questions go here - below the lines:

========================================================


YOUR Answers:

- Q1: The minimum required depth limit for r6 to output its paths is 5

- Q2: breadth_first_bfs and depth_limited_search

- Q3: all algorithms (breadth_first_bfs, depth_first_recursive_search, depth_limited_search, and iterative_deepening_search)

# Part II - Informed Search & Heuristics

In <b>Part II</b> of this Jupyter Notebook, you apply informed search algorithms on 8-puzzle problem.

Remember how I tried to [solve it online](http://www.tilepuzzles.com/default.asp?p=12) during the lecture? Now, Artificial Intelligence will help solve this puzzle! 

The state of the puzzle are represented as a list of integers. 0 represents the empty position. 

> The following is an example start state. The blank is represented by a 0 digit.

In [10]:
startState = (1, 0, 3, 4, 2, 5, 6, 7, 8)

## <font color=orange> 8 Puzzle Dependencies

In [20]:
class EightPuzzle(Problem):
    """ The problem of sliding tiles numbered from 1 to 8 on a 3x3 board,
    where one of the squares is a blank, trying to reach a goal configuration.
    A board state is represented as a tuple of length 9, where the element at index i 
    represents the tile number at index i, or 0 if for the empty square, e.g. the goal:
        1 2 3
        4 5 6 ==> (1, 2, 3, 4, 5, 6, 7, 8, 0)
        7 8 _
    """

    def __init__(self, initial, goal=(0, 1, 2, 3, 4, 5, 6, 7, 8)):
        assert inversions(initial) % 2 == inversions(goal) % 2 # Parity check
        self.initial, self.goal = initial, goal
    
    def actions(self, state):
        """The indexes of the squares that the blank can move to."""
        moves = ((1, 3),    (0, 2, 4),    (1, 5),
                 (0, 4, 6), (1, 3, 5, 7), (2, 4, 8),
                 (3, 7),    (4, 6, 8),    (7, 5))
        blank = state.index(0)
        return moves[blank]
    
    def result(self, state, action):
        """Swap the blank with the square numbered `action`."""
        s = list(state)
        blank = state.index(0)
        s[action], s[blank] = s[blank], s[action]
        return tuple(s)
    
    def h1(self, node):
        """The misplaced tiles heuristic."""
        return hamming_distance(node.state, self.goal)
    
    def h2(self, node):
        """The Manhattan heuristic."""
        X = (0, 1, 2, 0, 1, 2, 0, 1, 2)
        Y = (0, 0, 0, 1, 1, 1, 2, 2, 2)
        return sum(abs(X[s] - X[g]) + abs(Y[s] - Y[g])
                   for (s, g) in zip(node.state, self.goal) if s != 0)
    
    def h3(self, node): 
        """The Manhattan heuristic. (plus square root)"""
        X = (0, 1, 2, 0, 1, 2, 0, 1, 2)
        Y = (0, 0, 0, 1, 1, 1, 2, 2, 2)
        return (sum(abs(X[s] - X[g]) + abs(Y[s] - Y[g])
                   for (s, g) in zip(node.state, self.goal) if s != 0)) ** 1/2
    
    def h4(self, node): return max(self.h1(node), self.h2(node))
    
    def h(self, node): return self.h2(node)
    
    
def hamming_distance(A, B):
    "Number of positions where vectors A and B are different."
    return sum(a != b for a, b in zip(A, B))
    

def inversions(board):
    "The number of times a piece is a smaller number than a following piece."
    return sum((a > b and a != 0 and b != 0) for (a, b) in combinations(board, 2))
    
    
def board8(board, fmt=(3 * '{} {} {}\n')):
    "A string representing an 8-puzzle board"
    return fmt.format(*board).replace('0', '_')

class Board(defaultdict):
    empty = '.'
    off = '#'
    def __init__(self, board=None, width=8, height=8, to_move=None, **kwds):
        if board is not None:
            self.update(board)
            self.width, self.height = (board.width, board.height) 
        else:
            self.width, self.height = (width, height)
        self.to_move = to_move

    def __missing__(self, key):
        x, y = key
        if x < 0 or x >= self.width or y < 0 or y >= self.height:
            return self.off
        else:
            return self.empty
        
    def __repr__(self):
        def row(y): return ' '.join(self[x, y] for x in range(self.width))
        return '\n'.join(row(y) for y in range(self.height))
            
    def __hash__(self): 
        return hash(tuple(sorted(self.items()))) + hash(self.to_move)

## <font color=green> 8 Puzzle Tests

> Define a new `EightPuzzle` problem named `e6` with `initial` as `startState`.

In [21]:
e6 = EightPuzzle(startState)

> Solve `e6` using `weighted_astar_search` and show the solution path with `path_states()` function.

In [22]:
for s in path_states(weighted_astar_search(e6)):
    print(board8(s))

1 _ 3
4 2 5
6 7 8

1 2 3
4 _ 5
6 7 8

1 2 3
_ 4 5
6 7 8

_ 2 3
1 4 5
6 7 8

2 _ 3
1 4 5
6 7 8

2 3 _
1 4 5
6 7 8

2 3 5
1 4 _
6 7 8

2 3 5
1 _ 4
6 7 8

2 _ 5
1 3 4
6 7 8

_ 2 5
1 3 4
6 7 8

1 2 5
_ 3 4
6 7 8

1 2 5
3 _ 4
6 7 8

1 2 5
3 4 _
6 7 8

1 2 _
3 4 5
6 7 8

1 _ 2
3 4 5
6 7 8

_ 1 2
3 4 5
6 7 8



## Unsolvable 8-Puzzle Problems

There are cases in which the [8 puzzle is not solvable](https://www.cs.princeton.edu/courses/archive/fall12/cos226/assignments/8puzzle.html). For example, in the simple example below, switching the position of 1 and 2 makes it impossible to reach the goal state. It would seem that this could be done with a few simple slides, but tests with several algorithms show that they cannot find a solution.

In general, an odd number of inversions ([inverted number positions](https://www.geeksforgeeks.org/check-instance-8-puzzle-solvable/)) from the starting state in comparision to the goal state leads to an unsolvable puzzle.

In [23]:
unsolvable_startState = (2, 1, 3, 4, 5, 6, 7, 8, 0)

> Try to define a new `EightPuzzle` problem named `e7` using `unsolvable_startState`.

In [24]:
e7 = EightPuzzle(unsolvable_startState)

# AssertionError

AssertionError: 

## Heuristic Functions: h3 and h4

Add two new heuristics `h3` and `h4` to `EightPuzzle` class. `h3` should be the squared root of Manhattan distance and `h4` should be defined as the maximum of `h1` and `h2`. Try running the `weighted_astar_search` on `e6` once with `h3` and once with `h4`. They should run error-free ---we will test your code!

## Comparing Heuristics

Similar to "Comparing heuritic" report in `search4e.ipynb` and using `CountCalls` class, generate a report to compare the results of A* using the four heuristics `h1`, `h2`, `h3`, and `h4` on `e6`.

## <font color=orange> Comparing Heuristics Dependencies

In [25]:
class CountCalls:
    """Delegate all attribute gets to the object, and count them in ._counts"""
    def __init__(self, obj):
        self._object = obj
        self._counts = Counter()
        
    def __getattr__(self, attr):
        "Delegate to the original object, after incrementing a counter."
        self._counts[attr] += 1
        return getattr(self._object, attr)

        
def report(searchers, problems, verbose=True):
    """Show summary statistics for each searcher (and on each problem unless verbose is false)."""
    for searcher in searchers:
        print(searcher.__name__ + ':')
        total_counts = Counter()
        for p in problems:
            prob   = CountCalls(p)
            soln   = searcher(prob)
            counts = prob._counts; 
            counts.update(actions=len(soln), cost=soln.path_cost)
            total_counts += counts
            if verbose: report_counts(counts, str(p)[:40])
        report_counts(total_counts, 'TOTAL\n')
        
def report_counts(counts, name):
    """Print one line of the counts report."""
    print('{:9,d} nodes |{:9,d} goal |{:5.0f} cost |{:8,d} actions | {}'.format(
          counts['result'], counts['is_goal'], counts['cost'], counts['actions'], name))

## <font color=green> Comparing Heuristics Report

In [26]:
def astar_misplaced_tiles(problem): return weighted_astar_search(problem, h=problem.h1)

def astar_manhattan(problem): return weighted_astar_search(problem, h=problem.h2)

def astar_sqrt_manhattan(problem): return weighted_astar_search(problem, h=problem.h3)

def astar_max(problem): return weighted_astar_search(problem, h=problem.h4)

report([astar_misplaced_tiles, astar_manhattan, astar_sqrt_manhattan, astar_max], [e6])

astar_misplaced_tiles:
      918 nodes |      340 goal |   15 cost |     354 actions | EightPuzzle((1, 0, 3, 4, 2, 5, 6, 7, 8),
      918 nodes |      340 goal |   15 cost |     354 actions | TOTAL

astar_manhattan:
      330 nodes |      123 goal |   15 cost |     137 actions | EightPuzzle((1, 0, 3, 4, 2, 5, 6, 7, 8),
      330 nodes |      123 goal |   15 cost |     137 actions | TOTAL

astar_sqrt_manhattan:
      836 nodes |      312 goal |   15 cost |     326 actions | EightPuzzle((1, 0, 3, 4, 2, 5, 6, 7, 8),
      836 nodes |      312 goal |   15 cost |     326 actions | TOTAL

astar_max:
      341 nodes |      127 goal |   15 cost |     141 actions | EightPuzzle((1, 0, 3, 4, 2, 5, 6, 7, 8),
      341 nodes |      127 goal |   15 cost |     141 actions | TOTAL



## Part II Questions

- Q4 [2 points] - Could you define `e7`? Explain why based on the provided resources in the links above.

- Q5 [8 points] - According to the report you generated, which heuristic is the best one among `[h1, h2, h3, h4]`? Explain COMPLETELY.

- Your answers to Part II questions go here - below the lines:

========================================================


YOUR Answers:

- Q4: I could not define e7 because the start state creates an unsolvable board. e7's board is unsolvable because it has an odd number of inversions and the goal board has an even number of inversions, where "an inversion is any pair of blocks i and j where i < j but i appears after j when considering the board in row-major order." A start state of: (2, 1, 3, 4, 5, 6, 7, 8, 0) has 1 inversion (2-1).

- Q5: The best heuristic is the manhattan heuristic since it explores the least amount of nodes, gets to the goal the fastest, and executes the fewest number of actions.

# Part III - GBFS and A*

The following graph is similar to the example you worked on as class activity. Answer the following questions. <font color=red> ALL your answers should be typed in including the math inequality. Handwritten answers or screenshots will get NO credit. You may use [Latex](https://oeis.org/wiki/List_of_LaTeX_mathematical_symbols) for math. You can put your math equations between `$` You may also put them between `$$` to align it on center</font>. See how Latex is used in the questions to display the math by putting them between `$$`

<img src="A-star.png" align="left"/>

## Part III Questions

- Q6- What solution and goal would be returned if you run Greedy Best-First Search on this graph? What is the total cost of solution?


- Q7- What solution and goal would be returned if you run A-star Search on this graph? What is the total cost of solution?


- Q8- Is this heuristic admissible? You must prove admissibility using the inequality from slides for every node or to prove otherwise you must show a counter-example.
$$\forall\, node\,n, h(n) \le h^*(n)$$
> where $h^*(n)$ is the true actual (minimal) cost from $n$ to goal



- Q9- Is this heuristic consistent? You must prove consistency using the inequality from slides for every node or to prove otherwise you must show a counter-example.
 a heuristic $h$ is consistent if for every node $n$ of a parent node $p$,

$$h(p) \le h(n) + \mathrm{stepcost}(p,n)$$

YOUR Answers Go HERE:

- Q6- S -> B -> E -> J -> G1 (total cost = 30)


- Q7- S -> A -> C -> F -> G1 (total cost = 35)


- Q8- Admissability Test: $\forall\, node\,n, h(n) \le h^*(n)$
      S: h(n) = 10, h*(n) = 30 ✓
      A: h(n) = 12, h*(n) = 30 ✓
      B: h(n) = 10, h*(n) = 20 ✓
      C: h(n) = 10, h*(n) = 20 ✓
      D: h(n) = 15, h*(n) = 15 ✓
      E: h(n) = 10, h*(n) = 10 ✓
      F: h(n) = 10, h*(n) = 10 ✓
      H: h(n) = 5, h*(n) = 10 ✓
      J: h(n) = 5, h*(n) = 5 ✓
      K: h(n) = 10, h*(n) = 10 ✓
      This heuristic is admissable


- Q9- Consistency Test: $h(p) \le h(n) + \mathrm{stepcost}(p,n)$
      S to A: h(p) = 10, h(n) = 12, stepcost(p,n) =5 ✓
      S to B: h(p) = 10, h(n) = 10, stepcost(p,n) = 10 ✓
      A to C: h(p) = 12, h(n) = 10, stepcost(p,n) = 10 ✓
      A to D: h(p) = 12, h(n) = 15, stepcost(p,n) = 10 ✓
      B to E: h(p) = 10, h(n) = 10, stepcost(p,n) = 10 ✓
      C to F: h(p) = 10, h(n) = 10, stepcost(p,n) = 10 ✓
      D to H: h(p) = 15, h(n) = 5, stepcost(p,n) = 10 ✓
      D to J: h(p) = 15, h(n) = 5, stepcost(p,n) = 10 ✓
      E to J: h(p) = 10, h(n) = 5, stepcost(p,n) = 5 ✓
      E to K: h(p) = 10, h(n) = 10, stepcost(p,n) = 5 ✓
      F to G1: h(p) = 10, h(n) = 0, stepcost(p,n) = 10 ✓
      H to G1: h(p) = 5, h(n) = 0, stepcost(p,n) = 10 ✓
      J to G1: h(p) = 5, h(n) = 0, stepcost(p,n) = 5 ✓
      K to G2: h(p) = 10, h(n) = 0, stepcost(p,n) = 10 ✓
      This heuristic is consistent

<font color=blue>Congratulations! </font>You finished the first assignment of AI class! You learned and praticed a lot of things that we discussed in class. This is one of the cool features of Jupyter Notebooks, you can have all the contents, text, codes and plots in an interactive environment. We are going to have similar assignments for the following sections and chapters. Stay tuned! 

## Grading

Assignment 1 has a maximum of 100 points. Make sure that you get the desired outputs for all cells that you implemented. Also, your notebook should be written with no grammatical and spelling errors and should be nicely-formatted and easy-to-read.

The breakdown of the 100 points is as follows:

Part I has 40 points:
- 30 points: codes and correct outputs
- 10 points: correct answer of the Part I question {Q1: 2 points Q2 & Q3: 4 points}

Part II has 35 points:
- 25 points: codes and correct outputs
- 10 points: correct answers of Part II questions{Q4: 2 points Q5: 8 points}

Part III has 25 points:
- 5 points: Greedy Best First Search (Q6)
- 5 points: A* (Q7)
- 5 points: Admissibility (Q8)
- 10 points: Consistency (Q9)

Follow the instructions of each part and section carefully.

<b>Up to 10 points will be deducted if your submitted notebook is not easy to read and follow or if it has grammatical and spelling errors.</b>

## How to Submit and Due Date

Name your notebook ```Lastname-A1.ipynb```.  So, for me it would be ```Vafaei-A1.ipynb```.  Submit ONLY the Jupyter Notebook file using the ```Assignment-1``` link on Blackboard. You should NOT include the `.png` images for the graphs, we don't need them!

Grading will be based on 

  * correct implementation, correct answer to the questions, and
  * readability of the notebook.
  
<font color=red><b>Due Date: Monday October 5th, 11:59PM.</b></font>

## References

- AI A Modern Approach - 4th Edition - Russel & Norvig Textbook